# Architectures

> Scripts to get architectures for the models

In [1]:
#| default_exp architectures

In [2]:
#| export
#| hide

import torch
import torch.nn as nn
from abc import ABC, abstractmethod
from torch import Tensor

## Sampling

In [3]:
#| export
class Sampling(nn.Module):
    def forward(self, z_mean: Tensor, z_log_var: Tensor) -> Tensor:
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std

## Conv5 Architecture

### Kernel Sizes: 5, 7 9, 13

In [4]:
#| export
class Conv5Encoder(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Flatten()
        )
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)
        )

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

In [5]:
#| export
class Conv5Decoder(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * self.seq_len),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, self.seq_len))
        )

        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding=6),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding=3),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, z: Tensor) -> Tensor:
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

In [6]:
#| export
def get_conv5_vae_components(seq_len, feat_dim, latent_dim, dropout_rate=0.2):
    """
    Returns an instance of Conv5Encoder and Conv5Decoder based on the given parameters.
    
    Args:
        seq_len (int): Length of input sequence.
        feat_dim (int): Dimensionality of input features.
        latent_dim (int): Dimensionality of the latent space.
        dropout_rate (float): Dropout rate to use in the model.
    
    Returns:
        encoder (Conv5Encoder): The encoder part of the VAE.
        decoder (Conv5Decoder): The decoder part of the VAE.
    """
    encoder = Conv5Encoder(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5Decoder(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

### Legit Tsgm

In [7]:
#| export
class Conv5EncoderLegitTsgm(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float = 0.2):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=10, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=4, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Flatten()
        )
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)  # Output z_mean and z_log_var
        )

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

In [8]:
#| export
class Conv5DecoderLegitTsgm(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float = 0.2):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim

        # Dense layers to upscale the latent dimensions
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * seq_len),  # Adjust output size based on sequence length
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, seq_len))
        )

        # Transpose Convolutional layers with calculated padding for "same" effect
        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim,
                               kernel_size=(10 if seq_len > 10 else seq_len), stride=1,
                               padding=(5 if seq_len > 10 else seq_len // 2)),
                               # Adjust padding based on actual sequence length
                               # This ensures the output shape matches the input shape
                               # If seq_len <= 10 use same padding as input size
                               # Adjust these parameters based on actual needs and testing
                               activation="sigmoid")

    def forward(self, z: Tensor) -> Tensor:
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

In [9]:
#| export
def get_conv5_legit_tsgm_vae_components(seq_len, feat_dim, latent_dim, dropout_rate=0.2):
    """
    Returns an instance of Conv5Encoder and Conv5Decoder based on the given parameters.
    
    Args:
        seq_len (int): Length of input sequence.
        feat_dim (int): Dimensionality of input features.
        latent_dim (int): Dimensionality of the latent space.
        dropout_rate (float): Dropout rate to use in the model.
    
    Returns:
        encoder (Conv5Encoder): The encoder part of the VAE.
        decoder (Conv5Decoder): The decoder part of the VAE.
    """
    encoder = Conv5EncoderLegitTsgm(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5DecoderLegitTsgm(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()